# Q-Learning in Python

This notebook implements Q-Learning for tabular environments.

**Notebook created by [Víctor Campos](https://imatge.upc.edu/web/people/victor-campos) and [Xavier Giro-i-Nieto](https://imatge.upc.edu/web/people/xavier-giro)for the [Postgraduate course in Artificial Intelligence with Deep Learning](https://www.talent.upc.edu/ing/estudis/formacio/curs/310400/postgrau-artificial-intelligence-deep-learning/) in [UPC School](https://www.talent.upc.edu/ing/) (2020).**

![Víctor Campos](https://imatge.upc.edu/web/sites/default/files/styles/medium/public/users/vcampos//photo.jpg?itok=eCtqXNX9)

![Xavier Giro-i-Nieto](https://telecombcn-dl.github.io/2019-dlcv/img/instructors/XavierGiro-160x160.jpg)

## Import dependencies

We will use OpenAI Gym to simulate the environment and numpy to perform computations. 

In [0]:
import gym
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

## Visualize the environment

We will train an agent in the [FrozenLake-v0](https://gym.openai.com/envs/FrozenLake-v0/) environment. Read the documentation and make sure that you understand the structure of the problem before moving forward.

**Exercise #1.** Visualize a rollout of a random agent in the environment. Use the [documentation](http://gym.openai.com/docs/) for OpenAI Gym as a reference.

In [0]:
# Create an instance of the environment
env = gym.make("FrozenLake-v0")

# TODO: reset the environment
env.TODO
# Allow a maximum of 10 interactions
for t in range(10):
  print("\nTimestep {}".format(t))
  env.render()
  # TODO: sample a random action
  a = env.TODO
  # TODO: simulate the action in the environment; 
  #       make sure to capture the 'done' signal
  ob, r, done, _ = env.TODO
  # Exit if the episode terminated
  if done:
    print("\nEpisode terminated early")
    break

## Tabular Q-Learning Agent

We will now implement an agent that performs Q-Learning in the tabular setting. It will maintain a table of Q(s,a) values, with shape `num_states x num_actions`, that will be updated online with the stream of experience collected by interacting with the environment.

Exploration is critical in RL. The agent needs to continuously try actions that might seem suboptimal given its current beliefs in order to avoid getting trapped in local optima. We will use $\epsilon$-greedy exploration for this purpose: a strategy that will sample random actions with probability $\epsilon$, or will act greedily otherwise. We will decay $\epsilon$ through the course of training, starting with a high value that favors exploration and slowly transitioning towards a more greedy policy.

**Exercise #2.** Implement `QLearningAgent.greedy_action()`, a function that returns $\text{argmax}_aQ(s,a)$.

**Exercise #3.** Implement `QLearningAgent.eps_greedy_action()`, a function that returns a random action with probability $\epsilon$ or $\text{argmax}_aQ(s,a)$ otherwise.

**Exercise #4.** Implement `QLearningAgent.update_q_values()`, a function that receives a tuple $(s_t,a_t,r_t,s_{t+1},d)$ and performs a TD update to the table of Q values. Pay special attention to the computation of the TD target for the last step of an episode (when `done==True`).

In [0]:
class QLearningAgent:
  """Tabular Q-Learning Agent with epsilon-greedy exploration."""
  def __init__(self, env_id, step_size=0.5, gamma=0.99,
               init_eps=1.0, final_eps=0.05, eps_decay_steps=50000):
    # Use separate env instances for training and testing
    self.train_env = gym.make(env_id)
    self.test_env = gym.make(env_id)

    # Step size (this plays a similar role to the learning rate in SGD)
    self.step_size = step_size

    # Discount factor
    self.gamma = gamma

    # Epsilon, for epsilon-greedy exploration
    self.eps = init_eps
    self.init_eps = init_eps
    self.final_eps = final_eps
    self.eps_decay_steps = eps_decay_steps
    self.eps_delta = (self.final_eps - self.init_eps) / self.eps_decay_steps

    # Table of Q-values, initialized to zero
    self.q = np.zeros(
        (self.train_env.observation_space.n, self.train_env.action_space.n))
    
    # Keep track of the current state of the training env
    self.s = self.train_env.reset()

  def update_eps(self):
    """Update the value of epsilon, ensuring that self.eps>=self.final_eps."""
    self.eps = max(self.eps + self.eps_delta, self.final_eps)
  
  def greedy_action(self, s):
    # TODO: Returns argmax_a Q(s,a)
    return np.argmax(TODO)

  def eps_greedy_action(self, s):
    # TODO: Returns random action with prob self.eps, or greedy action otherwise.
    if np.random.random() < self.eps:
      return self.TODO
    else:
      return self.TODO

  def update_q_values(self, s, a, r, next_s, done):
    # TODO: Given a transition (s, a, r, s', done), perform a TD update to Q(s,a).
    if done:
      target = TODO
    else:
      target = TODO
    self.q[s, a] += self.step_size * (target - self.q[s, a])

  def perform_train_step(self):
    """Performs one RL interaction and updates the Q-values."""
    # Act epsilon-greedily
    a = self.eps_greedy_action(self.s)
    next_s, r, done, _ = self.train_env.step(a)

    # Update table of Q values
    self.update_q_values(self.s, a, r, next_s, done)

    # Reset the env if the episode terminated
    if done:
      self.s = self.train_env.reset()
    else:
      self.s = next_s
    
    # Update epsilon
    self.update_eps()

  def test(self, render=False):
    """Perform an evaluation rollout with the greedy policy.
    Returns the cumulative reward."""
    s = self.test_env.reset()
    done = False
    cumulative_r = 0.
    while not done:
      if render:
        self.test_env.render()
      s, r, done, _ = self.test_env.step(self.greedy_action(s))
      cumulative_r += r
    return cumulative_r

## Training loop

We are now ready to train the agent. We will track the performance of the agent by performing evaluation rollouts periodically. In order to account for the stochasticity of the environment, the mean over several evaluation episodes is reported.

**Exercise #5.** Train the agent with different hyperparameter configurations. Which ones have a larger influence in the results? 

In [0]:
NUM_TRAINING_STEPS = 100000
EVALUATION_FREQ = 100
NUM_EVALUATION_EPISODES = 20

agent = QLearningAgent("FrozenLake-v0", 
                       step_size=0.1, 
                       gamma=0.99,
                       init_eps=1.0, 
                       final_eps=0.1, 
                       eps_decay_steps=NUM_TRAINING_STEPS)

iter_history, rew_history = [], []
for iter_idx in tqdm(range(NUM_TRAINING_STEPS)):
  agent.perform_train_step()
  if iter_idx % EVALUATION_FREQ == 0 or iter_idx == (NUM_TRAINING_STEPS - 1):
    rew = np.mean([agent.test() for _ in range(NUM_EVALUATION_EPISODES)])
    iter_history.append(iter_idx + 1)
    rew_history.append(rew)

# Plot results
fig, ax = plt.subplots(1, 1, figsize=(9,4))
ax.plot(iter_history, rew_history, label="Agent's reward")
ax.plot(iter_history, 
        [agent.train_env.spec.reward_threshold] * len(iter_history),
        'r--', label="Maximum reward")
ax.set_xlabel("Environment steps")
ax.set_ylabel("Reward")
_ = ax.legend()

## Visualizing the learned policy

In [0]:
agent.test(render=True)